In [27]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\thoma\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\thoma\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\thoma\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\thoma\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [28]:
import pandas as pd
import numpy as np
from matplotlib import pyplot
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import LabelEncoder, StandardScaler
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_recall_curve,f1_score,auc
from sklearn.metrics import confusion_matrix, classification_report

In [29]:
# Set seed for reproduceable results
np.random.seed(500)
# Read in to Pandas DataFrame and drop the first row(which contained column names as I have assigned new names)
reviews = pd.read_csv(r"rt_reviews.csv", names = ['target', 'review'], encoding = 'latin-1')
reviews = reviews.iloc[1:]
reviews = reviews.iloc[:480000] # Choose num here

C:\Users\thoma\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [30]:
for i in range(reviews.shape[0]):
    #if(isinstance(reviews['target'].values[i],)):
    if(reviews['target'].values[i] == '0'):
        reviews['target'].values[i] = 0
    elif(reviews['target'].values[i] == '1'):
        reviews['target'].values[i] = 1
for i in range(reviews.shape[0]):
    if(isinstance(reviews['target'].values[i],str)):
        print("Caught: ",reviews['target'].values[i])
    if(reviews['target'].values[i] != 0 and reviews['target'].values[i] != 1):
        print("Error")

In [31]:

# Data is of shape(480000,2) The first column is the target, the second in the review

# Make data smaller for ease of implementation
reviews = reviews.iloc[:480000] # Choose num here

In [32]:
# Remove and row where either datafield is blank, no rows contained blank data so the shape remains 480000,2 
reviews.dropna(inplace = True)
reviews.shape

(480000, 2)

In [33]:
# Convert all text to lowercase so all words recognized the same 
reviews['review'] = [entry.lower() for entry in reviews['review']]

# Can now been seen to contain no capital letters
reviews.head()
#type(reviews)


,target,review
1,1,"manakamana doesn't answer any questions, yet ..."
2,1,wilfully offensive and powered by a chest-thu...
3,0,it would be difficult to imagine material mor...
4,0,despite the gusto its star brings to the role...
5,0,if there was a good idea at the core of this ...


In [34]:
# Tokenize each review: this process converts each review into a set of words. 
reviews['review'] = [word_tokenize(entry) for entry in reviews['review']]
type(reviews)

pandas.core.frame.DataFrame

In [36]:
reviews.head()

,target,review
1,1,"[manakamana, does, n't, answer, any, questions..."
2,1,"[wilfully, offensive, and, powered, by, a, che..."
3,0,"[it, would, be, difficult, to, imagine, materi..."
4,0,"[despite, the, gusto, its, star, brings, to, t..."
5,0,"[if, there, was, a, good, idea, at, the, core,..."


In [37]:
# Next steps are stemming/lemmenting which reduces words to their root so that words such as 'loving' and 'loved'
# both reduce to 'love'

# Creating tags so that lemmatizer can understand verbs from nouns from adjectives 
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV

for index,entry in enumerate(reviews['review']):
    index = index+1 # Index seems to off by one, this fixes it
    # Words that follow the rules will end up in this list
    Final_words = []
    
    word_Lemmatized = WordNetLemmatizer()
    
    for word, tag in pos_tag(entry):
        if word not in stopwords.words('english') and word.isalpha():
            word_Final = word_Lemmatized.lemmatize(word, tag_map[tag[0]])
            Final_words.append(word_Final)
        reviews.loc[index, 'text_final'] = str(Final_words)

In [38]:
reviews.shape

(480000, 3)

In [39]:
reviews.tail() 

,target,review,text_final
479996,0,"[zemeckis, seems, unable, to, admit, that, the...","['zemeckis', 'seem', 'unable', 'admit', 'motio..."
479997,1,"[movies, like, the, kids, are, all, right, --,...","['movie', 'like', 'kid', 'right', 'beautifully..."
479998,0,"[film-savvy, audiences, soon, will, catch, ont...","['audience', 'soon', 'catch', 'onto', 'winterb..."
479999,1,"[an, odd, yet, enjoyable, film, .]","['odd', 'yet', 'enjoyable', 'film']"
480000,1,"[no, other, animation, studio, ,, even, our, b...","['animation', 'studio', 'even', 'beloved', 'pi..."


In [40]:
# Create Train and Test splits 
test_s = .25
train_x, test_x, train_y, test_y = model_selection.train_test_split(reviews['text_final'], reviews['target'], test_size=test_s)

In [41]:
print("\ntrain_x Shape: ", train_x.shape)
print("\ntrain_y Shape: ", train_y.shape)
print("\ntest_x Shape: ", test_x.shape)
print("\ntest_y Shape: ", test_y.shape)
print(test_x)


train_x Shape:  (360000,)

train_y Shape:  (360000,)

test_x Shape:  (120000,)

test_y Shape:  (120000,)
360649    ['intimation', 'abu', 'ghraib', 'allusion', 'w...
285349    ['occasionally', 'stir', 'peek', 'place', 'phe...
59650     ['glum', 'muddy', 'muddle', 'gory', 'tell', 'b...
178735    ['repo', 'men', 'set', 'put', 'motion', 'premi...
84349     ['struggle', 'find', 'balance', 'contemporary'...
                                ...                        
429064    ['conventional', 'storytelling', 'far', 'less'...
223840    ['know', 'young', 'beautiful', 'hip', 'could',...
338243    ['fun', 'science', 'fiction', 'action', 'film'...
420399    ['david', 'russell', 'film', 'fizz', 'energy',...
292302    ['fool', 'title', 'bouncy', 'comedy', 'tonally...
Name: text_final, Length: 120000, dtype: object


In [42]:
Encoder = LabelEncoder()
train_y = Encoder.fit_transform(train_y)
test_y = Encoder.fit_transform(test_y)

In [ ]:
max_f = 10000
Tfidf_vect = TfidfVectorizer(max_features=max_f)
Tfidf_vect.fit(reviews['text_final'])
train_x_Tfidf = Tfidf_vect.transform(train_x)
test_x_Tfidf = Tfidf_vect.transform(test_x)
print(Tfidf_vect.vocabulary_)

In [44]:
print(train_x_Tfidf)

  (0, 8673)	0.24114998547474012
  (0, 8337)	0.2758537506369254
  (0, 7948)	0.20689398482346452
  (0, 7336)	0.26126901320769347
  (0, 7112)	0.24557843103987745
  (0, 5076)	0.3518463599655684
  (0, 4227)	0.2515952044714167
  (0, 4185)	0.38844645665124905
  (0, 3796)	0.15917919526995308
  (0, 3336)	0.19395978614723236
  (0, 3281)	0.3058363629916194
  (0, 3190)	0.21311210728582683
  (0, 2132)	0.22096420787578103
  (0, 1636)	0.17444563207480088
  (0, 303)	0.27426597560488863
  (1, 9018)	0.9204590189459473
  (1, 8996)	0.1847324547643638
  (1, 6794)	0.21140368794485123
  (1, 6553)	0.1332403280894473
  (1, 4415)	0.1585775172860864
  (1, 2684)	0.17617542763708552
  (2, 8180)	0.3056480339958997
  (2, 7636)	0.3113245314385503
  (2, 5471)	0.33623796881690776
  (2, 5207)	0.21967126334406667
  :	:
  (359995, 1363)	0.20051537223703397
  (359996, 9957)	0.4550029497266634
  (359996, 8220)	0.6035883353923267
  (359996, 2903)	0.49616807065821894
  (359996, 2577)	0.4271658726743701
  (359997, 8244)	0.3291

In [ ]:
# First Attempt at classifier is Naive Bays 
nbm = naive_bayes.MultinomialNB()
nbm.fit(train_x_Tfidf, train_y)

#Gaussian Naive Bayes
nbg = naive_bayes.GaussianNB()
#nbg.fit(train_x_Tfidf.toarray(), train_y)

#Bernoulli Naive Bayes
nbb = naive_bayes.BernoulliNB()
nbb.fit(train_x_Tfidf, train_y)

# predict the target on validation data
pred_nbm = nbm.predict(test_x_Tfidf)
#pred_nbg = nbg.predict(test_x_Tfidf.toarray())
pred_nbb = nbb.predict(test_x_Tfidf)

# output accuracy just to show it works
print("NB Multinomial: Accuracy: ", accuracy_score(pred_nbm, test_y))
#print("NB Gaussian: Accuracy: ", accuracy_score(pred_nbg, test_y))
print("NB Bernoulli: Accuracy: ", accuracy_score(pred_nbb, test_y))

svm = SVC(probability=True)
svm.fit(train_x_Tfidf, train_y)
pred_svm = svm.predict(test_x_Tfidf)
print("SVM: Accuracy: ", accuracy_score(pred_svm, test_y))


In [ ]:
#Analysis

#Predict Probability
nbm_probs = nbm.predict_proba(test_x_Tfidf)
nbm_probs = nbm_probs[:,1]
#nbg_probs = nbg.predict_proba(test_x_Tfidf.toarray())
#nbg_probs = nbg_probs[:,1]
nbb_probs = nbb.predict_proba(test_x_Tfidf)
nbb_probs = nbb_probs[:,1]
svm_probs = svm.predict_proba(test_x_Tfidf)
svm_probs = svm_probs[:,1]

#Classification Report
print(classification_report(test_y, pred_nbm, labels=[0,1]))
#print(classification_report(test_y, pred_nbg, labels=[0,1]))
print(classification_report(test_y, pred_nbb, labels=[0,1]))
print(classification_report(test_y, pred_svm, labels=[0,1]))

#Calculate precision-recall
precision_nbm, recall_nbm, thresholds_nbm = precision_recall_curve(test_y, nbm_probs)
#precision_nbg, recall_nbg, thresholds_nbg = precision_recall_curve(test_y, nbg_probs)
precision_nbb, recall_nbb, thresholds_nbb = precision_recall_curve(test_y, nbb_probs)
precision_svm, recall_svm, thresholds_svm = precision_recall_curve(test_y, svm_probs)

#Calculate F1
f1_nbm = f1_score(test_y, pred_nbm)
#f1_nbg = f1_score(test_y, pred_nbg)
f1_nbb = f1_score(test_y, pred_nbb)
f1_svm = f1_score(test_y, pred_svm)

#Calculate precision recal auc
# auc_nb = auc(recall_nb, precision_nb)
# auc_svm = auc(recall_svm, precision_svm)

# summarize scores
print("Test Split: ", test_s)
print("Max Features: ", max_f)
print('Naive-Bayes-Multinomial: f1=%.3f' % (f1_nbm))
print("Accuracy: ", accuracy_score(pred_nbm, test_y))
#print('Naive-Bayes-Gaussian: f1=%.3f' % (f1_nbg))
#print("Accuracy: ", accuracy_score(pred_nbg, test_y))
print('Naive-Bayes-Bernoulli: f1=%.3f' % (f1_nbb))
print("Accuracy: ", accuracy_score(pred_nbb, test_y))
print('Support Vector Machine: f1=%.3f' % (f1_svm))
print("Accuracy: ", accuracy_score(pred_svm, test_y))


# plot the precision-recall curves
no_skill = len(test_y[test_y==1]) / len(test_y)
pyplot.plot([0, 1], [no_skill, no_skill], linestyle='--', label='No Skill')
pyplot.plot(recall_nbm, precision_nbm, marker='.', label='Naive-Bayes Multinomial')
#pyplot.plot(recall_nbg, precision_nbg, marker='.', label='Naive-Bayes Gaussian')
pyplot.plot(recall_nbb, precision_nbb, marker='.', label='Naive-Bayes Bernoulli')
pyplot.plot(recall_svm, precision_svm, marker='.', label='SVM')

# axis labels
pyplot.xlabel('Recall')
pyplot.ylabel('Precision')

# show the legend
pyplot.legend()

# show the plot
pyplot.show()